In [14]:
import torch 
torch.cuda.is_available()

True

In [2]:
models = [torch.nn.Linear(4, 5) for i in range(3)]

In [3]:
import common.layers as layers

In [4]:
ens = layers.Ensemble(models)

In [10]:
ens.vmap

<function torch._functorch.apis.vmap.<locals>.wrapped(*args, **kwargs)>

In [13]:
ens.vmap()

ValueError: vmap({})(<inputs>): got no inputs. Maybe you forgot to add inputs, or you are trying to vmap over a function with no inputs. The latter is unsupported.